# Выдал метрику 384...
## Валидация: 43.34
Отобрал сильно корелированые фичи на основе хитмапа. Их всего 3, улучшило метрику не сильно где то на 0.02
Добавили фичей на основе этих трёх, дало прирост 8, неплохо
Перебрал гиперпараметр и использовал кросс валидацию, что дало минимальный прирост около 0.01
### Вывод: линейная модель как не крути хуже чем более актуальные модели,типа деревьев

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor
from src.const import PATH_TO_DATA, RANDOM_STATE
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


In [31]:
from src.utils import load_data
df_x_train, df_y_train, df_test = load_data()

In [32]:
columns_to_drop = [4, 10, 11]  

df_x_train = df_x_train[columns_to_drop]
df_test = df_test[columns_to_drop]

In [33]:
# Создадим новые признаки

# 1) Полиномиальные признаки (включая взаимодействия, степень=2)
poly = PolynomialFeatures(degree=2, include_bias=False)
# Применяем только к обучающей части
X_poly_train = poly.fit_transform(df_x_train)
X_poly_test = poly.transform(df_test)

# Полиномиальные фичи генерируются автоматически, но можно добавить и ручные:
# Логарифмы, корень, обратные величины:
log_train = np.log1p(df_x_train.clip(lower=0) + 1)  # log(x+1)
sqrt_train = np.sqrt(df_x_train.clip(lower=0))      # sqrt(x)
inv_train = 1 / (df_x_train.clip(lower=1))          # 1/(x+1), чтобы не было деления на 0

log_test = np.log1p(df_test.clip(lower=0) + 1)
sqrt_test = np.sqrt(df_test.clip(lower=0))
inv_test = 1 / (df_test.clip(lower=1))

# Соединяем всё вместе в один массив/фрейм
# Обратите внимание: X_poly_train и log_train - это numpy и pandas, их нужно собрать воедино
df_x_train = np.concatenate([
    X_poly_train,
    log_train.values,
    sqrt_train.values,
    inv_train.values
], axis=1)

df_test = np.concatenate([
    X_poly_test,
    log_test.values,
    sqrt_test.values,
    inv_test.values
], axis=1)

In [34]:
df_x_train

array([[0.        , 0.17575848, 2.20710049, ..., 1.        , 1.        ,
        0.45308313],
       [1.        , 6.85149107, 2.47917775, ..., 1.        , 0.14595363,
        0.40335954],
       [1.        , 6.00267265, 0.78072948, ..., 1.        , 0.16659246,
        1.        ],
       ...,
       [0.        , 8.08467479, 0.25196239, ..., 1.        , 0.12369081,
        1.        ],
       [1.        , 5.83825411, 2.51594822, ..., 1.        , 0.17128408,
        0.39746446],
       [1.        , 2.17421812, 0.347862  , ..., 1.        , 0.45993545,
        1.        ]])

In [35]:
X = df_x_train.copy()
y = df_y_train[0].values  # или df_y_train.squeeze() если это Series

# Разбиваем данные на обучающую и валидационную выборки
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y, 
    test_size=0.2,
    random_state=42
)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', Ridge(random_state=42))
])

cv = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)

param_grid = {
    'ridge__alpha': [0.01, 0.1, 1.0, 10.0, 100.0]
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print("Лучшие параметры:", grid_search.best_params_)
print("Лучшее отрицательное MSE (на кросс-валидации):", grid_search.best_score_)

val_pred = grid_search.predict(X_val)
mse = mean_squared_error(y_val, val_pred)
print(f"MSE на валидации: {mse:.4f}")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Лучшие параметры: {'ridge__alpha': 100.0}
Лучшее отрицательное MSE (на кросс-валидации): -42.436581834304874
MSE на валидации: 43.3474


In [28]:
# model = Ridge(random_state=RANDOM_STATE, alpha=0.1)
# model.fit(X_train, y_train)
# 
# val_pred = model.predict(X_val)
# mse = mean_squared_error(y_val, val_pred)
# print(f"MSE на валидации: {mse:.4f}")

MSE на валидации: 43.3486


MSE на валидации: 48.37 оставлены все фичи в том числе не коррелированые
MSE на валидации: 48.3595 без добавления фич  
MSE на валидации: 43.3486 с добавлением фич
MSE на валидации: 43.3474 с кросс валидацией и перебором параметров

In [36]:
from src.utils import save_predictions

y_pred = grid_search.predict(df_test)
save_predictions(y_pred)

Предсказания сохранены в: predictions\predictions_7.csv


'predictions\\predictions_7.csv'